In [48]:
pip install --upgrade numexpr


   ---------------------------------------- 97.0/97.0 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: numexpr
    Found existing installation: numexpr 2.9.0
    Uninstalling numexpr-2.9.0:
      Successfully uninstalled numexpr-2.9.0
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.


In [41]:
import os
import pandas as pd

# Import

### Read data

In [47]:
#Create mapping
mapping_path = "03_Mapping/Vehicle_Type_Mapping.xlsx"
mapping = pd.read_excel(mapping_path)
mapping_dct = dict(zip(mapping['Key'], mapping['Mapping']))

#Read man counts
man_path = "01_Input/Manual_Counts"
man_data = []
man_loc = []

for file in os.listdir(man_path):
    #Read all excel files
    if file.endswith(".xlsx"):
        loc = file[0:3]
        data = pd.read_excel(os.path.join(man_path, file))
        data.rename(mapper=mapping_dct, axis=1, inplace=True)
        data['Timestamp'] = data['Timestamp'] + pd.to_timedelta(1, unit='s')
        data['Timestamp'] = data['Timestamp'].dt.floor('1min')
        #check if data for this loc already exists, then append or create new loc
        if loc in man_loc:
            index = man_loc.index(loc)
            man_data[index] = pd.concat([man_data[index], data], ignore_index=True)
        else:
            man_data.append(data)
            man_loc.append(loc)
            

        
#Read cam counts
cam_path = "01_Input/Camera_Counts"
cam_data = []
cam_loc = []

for file in os.listdir(cam_path):
    #Read all csv files
    if file.endswith(".csv"):
        loc = file[0:3]
        data = pd.read_csv(os.path.join(cam_path, file))
        data.rename(mapper=mapping_dct, axis=1, inplace=True)
        #Adjust format of timestamps
        data['Timestamp'] = pd.to_datetime(data['Timestamp'])
        if data['Timestamp'].dt.tz is not None:
            data['Timestamp'] = data['Timestamp'].dt.tz_convert(None)
        #check if data for this loc already exists, then append or create new loc
        if loc in cam_loc:
            index = cam_loc.index(loc)
            cam_data[index] = pd.concat([cam_data[index], data], ignore_index=True)
        else:
            cam_data.append(data)
            cam_loc.append(loc)
            
#Read local counts
local_path = "01_Input/Local_Counts"
local_data = []
local_loc = []

for file in os.listdir(local_path):
    #Read all excel files
    if file.endswith(".xlsx"):
        loc = file[0:3]
        data = pd.read_excel(os.path.join(local_path, file))
        data.rename(mapper=mapping_dct, axis=1, inplace=True)
        data['Timestamp'] = data['Timestamp'] + pd.to_timedelta(1, unit='s')
        data['Timestamp'] = data['Timestamp'].dt.floor('1min')
        #check if data for this loc already exists, then append or create new loc
        if loc in local_loc:
            index = local_loc.index(loc)
            local_data[index] = pd.concat([local_data[index], data], ignore_index=True)
        else:
            local_data.append(data)
            local_loc.append(loc)
            
#Read induction loop counts
inLoop_path = "01_Input/Induction_Loop_Counts"
inLoop_data = []
inLoop_loc = []

for file in os.listdir(inLoop_path):
    #Read all csv files
    if file.endswith(".csv"):
        loc = file[0:3]
        data = pd.read_csv(os.path.join(inLoop_path, file), encoding='latin1', sep=";")
        data.rename(mapper=mapping_dct, axis=1, inplace=True)
        #Adjust format of timestamps
        data['Timestamp'] = pd.to_datetime(data['Timestamp'], format="%d.%m.%Y %H:%M")
        #check if data for this loc already exists, then append or create new loc
        if loc in inLoop_loc:
            index = inLoop_loc.index(loc)
            inLoop_data[index] = pd.concat([inLoop_data[index], data], ignore_index=True)
        else:
            inLoop_data.append(data)
            inLoop_loc.append(loc)

### Filter data

In [48]:
man_data_fil_hour = []

#filter to only have data from complete hours
for df in man_data:
    df['Date'] = df['Timestamp'].dt.date
    df['Hour'] = df['Timestamp'].dt.hour
    grouped = df.groupby(['Date', 'Hour'])
    counts = grouped.size()
    incomplete_indices = counts[counts != 6].index
    man_data_fil_hour.append(df[~df.set_index(['Date', 'Hour']).index.isin(incomplete_indices)])
    #df.drop(['Date', 'Hour'], axis=1, inplace=True)

In [49]:
#Filtered version of cam_data only containing timeframes available in man_data
cam_data_fil = [None] * len(cam_data)

#Fill the list
index_man = 0
for loc in man_loc:
    if loc in cam_loc:
        index_cam = cam_loc.index(loc)
        cam_data_fil[index_cam] = pd.merge(cam_data[index_cam], 
                                           man_data[index_man]['Timestamp'], on='Timestamp', how='inner')
    index_man +=1
    
#Filtered version of local_data only containing timeframes available in man_data
local_data_fil = [None] * len(cam_data)

#Fill the list
index_man = 0
for loc in man_loc:
    if loc in local_loc:
        index_local = local_loc.index(loc)
        local_data_fil[index_local] = pd.merge(local_data[index_local], 
                                           man_data[index_man]['Timestamp'], on='Timestamp', how='inner')
    index_man +=1


#Version of cam_data_fil only containing data where data for entire hour are available
cam_data_fil_hour = [None] * len(cam_data)

#Version of local_data_fil only containing data where data for entire hour are available
local_data_fil_hour = [None] * len(cam_data)

#Filtered version of inLoop_data only containing timeframes available in man_data_fil
inLoop_data_fil_hour = [None] * len(inLoop_data)

#Fill both lists
index_man = 0
for loc in man_loc:
    if loc in cam_loc:
        index_cam = cam_loc.index(loc)
        cam_data_fil_hour[index_cam] = pd.merge(cam_data[index_cam], 
                                                man_data_fil_hour[index_man]['Timestamp'], on='Timestamp', how='inner')
    if loc in inLoop_loc:
        index_inLoop = inLoop_loc.index(loc)
        inLoop_data_fil_hour[index_inLoop] = pd.merge(inLoop_data[index_inLoop], 
                                                man_data_fil_hour[index_man]['Timestamp'], on='Timestamp', how='inner')
    if loc in local_loc:
        index_local = local_loc.index(loc)
        local_data_fil_hour[index_local] = pd.merge(local_data[index_local], 
                                                man_data_fil_hour[index_man]['Timestamp'], on='Timestamp', how='inner')
    index_man +=1

# Export

### Manual count timeframe

In [50]:
export_path = "02_Output/Manual_Count_Timeframe-10_Minute_Intervall/"

for i in range(len(man_data)):
    man_data[i].to_excel(export_path + man_loc[i] + "_man_data.xlsx")  
    if man_loc[i] in cam_loc:
        loc_index = cam_loc.index(man_loc[i])
        cam_data_fil[loc_index].to_excel(export_path + cam_loc[loc_index] + "_cam_data.xlsx")
    if man_loc[i] in local_loc:
        loc_index = local_loc.index(man_loc[i])
        local_data_fil[loc_index].to_excel(export_path + local_loc[loc_index] + "_local_data.xlsx")  

### Manual count timeframe - complete hours

In [55]:
export_path = "02_Output/Manual_Count_Timeframe-1_Hour_Intervall/"

for i in range(len(man_data)):
    man_data_fil_hour[i].to_excel(export_path + man_loc[i] + "_man_data.xlsx")  
    if man_loc[i] in cam_loc:
        cam_index = cam_loc.index(man_loc[i])
        cam_data_fil_hour[cam_index].to_excel(export_path + cam_loc[cam_index] + "_cam_data.xlsx")
    if man_loc[i] in local_loc:
        loc_index = local_loc.index(man_loc[i])
        local_data_fil_hour[loc_index].to_excel(export_path + local_loc[loc_index] + "_local_data.xlsx")
    if man_loc[i] in inLoop_loc:
        inLoop_index = inLoop_loc.index(man_loc[i])
        #Drop irrelevant columns
        relevant_columns = ["Timestamp", "Total"]
        inLoop_data_fil_hour[inLoop_index][relevant_columns].to_excel(export_path + inLoop_loc[inLoop_index] + "_inLoop_data.xlsx")